In [1]:
from keras.preprocessing.image import ImageDataGenerator

from keras.utils import np_utils
from keras.models import Sequential
from keras.utils import array_to_img, img_to_array, load_img
from keras.layers import *
from keras.callbacks import ModelCheckpoint, EarlyStopping

import os
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score

In [2]:
path = r"C:\\Users\\USER\\Desktop\\font_classifier" #원본 이미지 경로 C:\Users\USER\Desktop\font_classifier
copy_path = r"C:\\Users\\USER\\Desktop\\font_classifier_copy" #복사 및 분류된 이미지 경로
#font_classifier 안에 모든 글꼴 이미지들이 다 저장되어 있음 (따로 하위폴더 없음)
#노트북 경로

In [3]:
import splitfolders

splitfolders.ratio(path, output=copy_path, seed = 43, ratio = (.7,.2,.1))

Copying files: 0 files [00:00, ? files/s]

Copying files: 55860 files [16:54, 55.05 files/s]


In [12]:
#각 train, test, vali 폴더 안에서 글꼴별로 구별되어 있는 걸 풀어서 섞고, 글꼴별 폴더 삭제하기

import shutil
import os

font_list = ["01","02","03","04","05"]
folder_list = ["val","test","train"]

for folder in folder_list :


    for font in font_list :
        currentPath = copy_path + f'\\{folder}'+ f'\\{font}'

        trainFileList = os.listdir(currentPath)
        trainFileList = [x for x in trainFileList if 'png' in x]

        for image in trainFileList :
            try :
                shutil.move(currentPath + f'\\{image}', copy_path + f'\\{folder}' + f'\\{image}')
            except :
                pass
    
        os.rmdir(currentPath)


In [14]:
print(len(os.listdir(copy_path + '\\test')))
print(len(os.listdir(copy_path + '\\train')))
print(len(os.listdir(copy_path + '\\val')))

5591
39101
11171


In [ ]:
trainFileList = os.listdir(path)

In [ ]:
trainFileList = [x for x in trainFileList if 'png' in x]

In [ ]:
label_list = list(set(map(lambda x : x[:-7], trainFileList))) # ex. a_b_c <-- 이런 거로 폴더명 리스트 만들기

In [15]:
folder_list = ["val","test","train"]

In [17]:
import shutil
import os

for folder in folder_list :
    currentPath = copy_path + f'\\{folder}'

    trainFileList = os.listdir(currentPath)
    trainFileList = [x for x in trainFileList if 'png' in x]

    label_list = list(set(map(lambda x : x[:-7], trainFileList))) 
    
    try:
        for key in label_list: # 글자마다 폴더가 생김
            os.makedirs(currentPath + f'\\{key}')
    except:
        pass

    # 여기까지는 정상적으로 처리 됨.

    # font_classifier 폴더에 있던 마구자비로 섞여있던 이미지 데이터들이 train 폴더 안에 있는 글자 폴더별로 옮겨짐
    for file in trainFileList:
        label = file.split('.')[0]
        targetPath = currentPath + f'\\{label}'
        try :
            shutil.move(currentPath+f'\\{file}', targetPath+f'\\{file}')
        except:
            pass


#여기까지 자료 준비 완료 (글자별로 분류된 상태)

### 데이터 준비 완료 -----------

In [ ]:
#데이터 변형

currentPath = copy_path + f'\\train'  #val, test도 해야 함.

trainDataGen = ImageDataGenerator(rescale=1./255,
                                 rotation_range = 10,
                                 width_shift_range=0.1,
                                 height_shift_range=0.1,
                                 shear_range=0.1,
                                 zoom_range=0.1,
                                 horizontal_flip=False,
                                 vertical_flip=False,
                                 fill_mode='nearest'
                                 )
# 밑에서 데이터를 가져옴.

In [ ]:
trainGenSet = trainDataGen.flow_from_directory(
    currentPath,
    batch_size=64,
    target_size=(28,28),
    class_mode='categorical'
)

Found 55860 images belonging to 11172 classes.


### 데이터 변형까지 완료

In [ ]:
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from skimage.transform import resize
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=10, shuffle=True)

In [ ]:
# 모델 구성
for train, test in kfold.split(inputs, targets):
    model = Sequential()
    model.add(Conv2D(64, kernel_size=(3,3), padding='same', input_shape=(28,28,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(128, kernel_size=(3,3),padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(256, kernel_size=(3,3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(18, activation='softmax'))
    
      # Compile the model
    model.compile(loss=loss_function,
                optimizer=optimizer,
                metrics=['accuracy'])

NameError: name 'inputs' is not defined

In [ ]:
# Fit data to model
history = model.fit(inputs[train], targets[train],
              batch_size=batch_size,
              epochs=no_epochs,
              verbose=verbosity)

In [ ]:
model.summary()